In [3]:
#corrigindo o 503
import pandas as pd
import json
import re
import time
from google import genai

client = genai.Client(api_key="key")

df = pd.read_excel("temp_redacoes_extraidas_adicional.xlsx", usecols="A:D")

PROMPT_BASE = """
Você é um profissional formado em Letras e é avaliador de redações do Exame Nacional do Ensino Médio (ENEM). Você irá avaliar cinco competências: (1) Demonstrar domínio da norma culta da língua escrita; (2) Compreender a proposta da redação e aplicar conceito das várias áreas de conhecimento para desenvolver o tema, dentro dos limites estruturais do texto dissertativo-argumentativo; (3) Selecionar, relacionar, organizar e interpretar informações, fatos, opiniões e argumentos em defesa de um ponto de vista; (4) Demonstrar conhecimento dos mecanismos linguísticos necessários para a construção da argumentação e (5) Elaborar a proposta de solução para o problema abordado, mostrando respeito aos valores humanos e considerando a diversidade sociocultural. Em cada competência a nota a ser atribuída é de 0 a 200. Faça um comentário geral sobre a redação avaliada e comentários específicos sobre a nota atribuída em cada competência.

Responda apenas com um JSON no seguinte formato:


{{
  "competencias": [
    {{
      "competencia": 1,
      "nota": <número inteiro entre 0 e 200>,
      "justificativa": "<comentário sobre a competência 1>"
    }},
    {{
      "competencia": 2,
      "nota": <número inteiro entre 0 e 200>,
      "justificativa": "<comentário sobre a competência 2>"
    }},
    {{
      "competencia": 3,
      "nota": <número inteiro entre 0 e 200>,
      "justificativa": "<comentário sobre a competência 3>"
    }},
    {{
      "competencia": 4,
      "nota": <número inteiro entre 0 e 200>,
      "justificativa": "<comentário sobre a competência 4>"
    }},
    {{
      "competencia": 5,
      "nota": <número inteiro entre 0 e 200>,
      "justificativa": "<comentário sobre a competência 5>"
    }}
  ],
  "nota_final": <soma das cinco notas>,
  "comentario_geral": "<comentário geral sobre a redação>"
}}

Texto da redação:
"TÍTULO: {titulo_redacao}
TEMA: {titulo_proposta}
TEXTO: {texto_redacao}"
"""

def extrair_json(texto):
    match = re.search(r'\{.*\}', texto, re.DOTALL)
    if match:
        json_text = match.group()
        try:
            return json.loads(json_text)
        except json.JSONDecodeError:
            return None
    return None

avaliacoes = []

for index, row in df.iterrows():
    prompt = PROMPT_BASE.format(
        titulo_redacao=row["Título da redação"],
        titulo_proposta=row["Título da proposta"],
        texto_redacao=row["Texto da redação"]
    )

    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt,
        )
        result = response.text.strip()

        # Remover delimitadores de markdown
        if result.startswith("```json"):
            result = result.strip("```json").strip("```").strip()

        # Tentativa de parsing
        data = extrair_json(result)

        if data is None:
            raise ValueError("Resposta não contém JSON válido.")

        # Adaptação para resposta com ou sem chave "avaliacao"
        if "avaliacao" in data:
            competencias = data["avaliacao"].get("competencias", [])
            nota_final = data["avaliacao"].get("nota_final")
            comentario_geral = data["avaliacao"].get("comentario_geral", "")
        elif "competencias" in data:
            competencias = data["competencias"]
            nota_final = data.get("nota_final")
            comentario_geral = data.get("comentario_geral", "")
        else:
            raise ValueError("Chave 'competencias' ou 'avaliacao' não encontrada.")

        if len(competencias) != 5:
            raise ValueError("Número incorreto de competências.")

        notas = [c["nota"] for c in competencias]
        justificativas = [c["justificativa"] for c in competencias]

        comentario_geral += "\n\n" + "\n".join([
            f"Competência {i+1}: {just}" for i, just in enumerate(justificativas)
        ])

        avaliacoes.append({
            "ID": row["ID"],
            "Comentário geral": comentario_geral,
            "Nota 1": notas[0],
            "Nota 2": notas[1],
            "Nota 3": notas[2],
            "Nota 4": notas[3],
            "Nota 5": notas[4],
            "Nota final": nota_final
        })

        print(f"✅ Redação {row['ID']} avaliada.")

    except Exception as e:
        print(f"❌ Erro ao avaliar a redação {row['ID']}: {e}")
        avaliacoes.append({
            "ID": row["ID"],
            "Comentário geral": f"Erro na avaliação: {e}",
            "Nota 1": None,
            "Nota 2": None,
            "Nota 3": None,
            "Nota 4": None,
            "Nota 5": None,
            "Nota final": None
        })

        with open(f"saidas/resposta_falha_redacao_{row['ID']}.txt", "w", encoding="utf-8") as f:
            f.write(result if 'result' in locals() else "Sem resposta da API.")

    time.sleep(1.5)

# Exporta planilha
df_out = pd.DataFrame(avaliacoes)
df_out.to_excel("avaliacoes_redacoes_adicionais.xlsx", index=False)
print("📁 Arquivo 'avaliacoes_redacoes_adicionais.xlsx' salvo com sucesso!")


✅ Redação 9 avaliada.
✅ Redação 20 avaliada.
✅ Redação 167 avaliada.
✅ Redação 355 avaliada.
✅ Redação 631 avaliada.
📁 Arquivo 'avaliacoes_redacoes_adicionais.xlsx' salvo com sucesso!
